# Multivariate pattern analysis of fMRI data to study category representation in the human brain

## In this notebook, we will work through a multivariate pattern analysis (MVPA) of one participant from Haxby et al. (2001)

The goal here is for you to learn about how to implement multivariate pattern analysis (MVPA), which attempts to determine whether distributed patterns of activity differ between conditions. In this case, we will be using machine learning to look at patterns of activaty in the visual system in response to different categories of images (e.g., faces, houses). This analysis will be done on a single participant from a landmark study: 

Haxby, J. V., Gobbini, M. I., Furey, M. L., Ishai, A., Schouten, J. L., & Pietrini, P. (2001). Distributed and overlapping representations of faces and objects in ventral temporal cortex. *Science*, 293, 2425-2430. http://dx.doi.org/10.1126/science.1063736

## Why are you having us analyze fMRI data in introductory cognitive neuroscience course?
I (Prof. Derek Huffman) believe that we learn best by actually working with the content from the field. For example, if you want to learn how to paint, sure, you can read a book about painting, you can look at paintings, you might even learn from watching videos of other people painting and lectures in which experienced artists teach you how to paint. However, you are going to learn best by actually picking up a paintbrush, dipping it in the paint, and trying to paint something yourself. You will learn how to mix colors together, how much pressure to apply to the canvas, which paintbrush to use in different situations, how long to let paint dry before adding different colors, and (perhaps most importantly) you will learn from your mistakes. Simply put, you will learn a lot more by practicing (as the saying goes: "experience is the best teacher"). Moreover, these practical experiences will help you learn more from your other resources. For example, you might revisit readings or discussions to really think about *why* we use certain techniques. Similarly, to *really* understand cognitive neuroscience, I believe that we have to actually practice some cognitive neuroscience, so pick up your paintbrush and start painting some brain maps with the Python code below!

## Attributions and citations for this notebook
Written by Professor Derek J. Huffman at Colby College, Waterville, ME 04901, USA (with the nilearn tutorials as a helpful starting point; Nilearn Contributors et al., 2023)

Most recent updates: 2024 JAN 18

Please keep the attribution line above as well as the citations below (update as you change the code):

Huffman, D. J. (2024). An in-depth exploration of the interplay between fMRI methods and theory in cognitive neuroscience. PsyArXiv.

Nilearn Contributors et al. (2023) nilearn. Available at: https://zenodo.org/record/8397156.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from nilearn.datasets import fetch_haxby
from nilearn.image import index_img
from nilearn.image import load_img
from nilearn.glm.first_level import FirstLevelModel
from nilearn.plotting import plot_design_matrix
from nilearn.plotting import plot_stat_map
from nilearn.plotting import show
from nilearn.plotting import view_img

Note: the 'from nilearn.glm.first_level import FirstLevelModel' will throw a warning about nilearn.glm being experimental (AKA we are on the cutting edge haha), but we can ignore this for now.

### Gather the fMRI and event/behavioral/timing data

Next, we will want to get all of the data. Here, we will primarily rely on the fetch_haxby() function. This function will load data from a single participant (subj2) to the location we set for datapath below.

#### First, define a variable with the path to the data:

In [ ]:
datapath = '.'

#### Then, use the fetch_haxby function to load the data:
Note: here we give the function the input of the datapath to let it know where to find the data.

In [ ]:
subject_data = fetch_haxby(datapath)

Now we can take a look at the subject_data.

In [ ]:
subject_data

So, what you might notice above is that for these nilearn datasets, it will essentially load strings that represent the location sof different files. You will note a few files of interest:

1) 'anat' = the anatomical / structural image of the participant's brain (T1 MRI)

2) 'func' = the functional image of the participant's brain (i.e., fMRI)

3) 'mask_vt' = a mask of the ventral temporal lobe, which is important for high-level vision

4) 'mask_face' = a mask of voxels that respond to faces (i.e., greater BOLD activity)

5) 'mask_house' = a mask of voxels that respond to houses (i.e., greater BOLD activity)

6) 'session_target' = the conditions of the dataset (here, labels = frame-by-frame indication of what was happening; chunks = the run or scan number from which that frame came... fMRI are gathered in runs, but the func file above is the concatenation or combination of all of those runs together).

In this particular case (for these nilearn datasets), the functional data can be accessed like this:

In [ ]:
fmri = subject_data.func[0]

Gather the labels for each frame. This information will be necessary for knowing what was happening on each frame of the experiment. The 'labels' column contains the task condition of each frame and the 'chunks' column contains the run information. These will be key features for training and testing the classifier (see below).

To load this file, we will use the pandas function read_csv. Note: a CSV file (comma-separated values) is one way of saving data in a file where things are separated by commas. Here, the file is separated by blank spaces, hence the sep=' ' (i.e., notice the blank space between the quotes). This function will create a pandas DataFrame, which is a convenient way of storing this type of data. The DataFrame is similar to a numpy.array, but allows for data with different types in different columns. Similar to a dictionary, you can access the columns with the following syntax, e.g., tr_labels['labels']

In [ ]:
tr_info = pd.read_csv(subject_data.session_target[0], sep=' ')
tr_info

## Now we need to set up the decoder (yes, machine learning, here we come!)

In [ ]:
from nilearn.decoding import Decoder
from sklearn.model_selection import LeaveOneGroupOut

Set up a function for printing the classification results.

In [ ]:
def print_class_accuracy(d):
    """
    A function for printing the classification results.
    
    Parameters
    ----------
    d : a fit decoder
        A model that was fit via nilearn
    
    Returns
    -------
    Out : None
        Prints the classification accuracy and chance level
    """
    classification_accuracy = np.mean(list(d.cv_scores_.values()))
    chance_level = 1. / len(d.classes_)
    print('Classifying {}'.format(d.classes_))
    print('Classification accuracy: {:.4f} / Chance level: {}'.format(
        classification_accuracy, chance_level))
    return classification_accuracy



### We will start by looking at the classification of faces vs. houses (i.e., related to our last assignment in that employed an activation analysis).

We will start by training and testing the classifier on individual frames of fMRI data. That is, we have 121 frames per run. Each block of trials is 24 seconds long, so we will be using 9 frames from each of those blocks per condition per run. For example, here you can see the labels for scissors for the first run.

In [ ]:
print(tr_info['labels'][0:20])

In [ ]:
fmri

In [ ]:
condition_mask = tr_info['labels'].isin(['face', 'house'])
conditions_sel = tr_info['labels'][condition_mask]
run_label = tr_info['chunks'][condition_mask]
fmri_sel = index_img(fmri, condition_mask)

# Confirm that we now have 2 conditions
print(conditions_sel.unique())

### And let's see how well a classifier can distinguish trials in which participants are viewing faces vs. trials in which they are viewing houses.

#### Some notes about the classifier

1) Here, we will use a support vector machine (SVM; estimator='svc'). The SVM is commonly used in fMRI because we have a lot of features (i.e., voxels) but a limited number of trials. Some machine learning classifiers will suffer from overfitting (i.e., not generalizing well to new data) because they have a lot of free parameters. The SVM does not change its number of parameters with more features, therefore it is a good technique for fMRI. More specifically, we will be using a linear SVM, which also has benefits with respect to interpretting the results. Typically, in the case of a linear classifier, we would say that if a region can classify between categories of stimuli, then it contains information about that category. This is a bit beyond the scope of what you need to know for this class, but with a nonlinear classifier, it is more difficult to know whether or not that region "contains the representation" (vs. other downstream regions, which would be able to "read this information out").

2) We will use an approach called *cross-validation*. This is a common approach in machine learning and it is the process of partitioning your data into a "training set" and a "test set." The basic idea is that you want to train and test your classifier on indepdent data to see how well it can generalize to new examples. Our measure of interest is the generalization performance on the test data. One thing to note in the context of fMRI data is that timepoints that come from the same run are *not* independent. For example, the slow timing of the hemodynamic response function makes nearby timepoints more similar to each other (i.e., the are correlated so they are not indepdendent). In a nutshell, it is best practice to train and test the classifier on data from different runs. So, here, we employ what is called leave-one-run-out cross-validation, where we train and test our classifier on data from each run. For example, on the first iteration, we would train our classifier on the data from runs 1 to 11 and then test the classifier on the data from run 0. The next iteration we would train the classifier on runs 0 and 2 to 11 and then test it on run 1, and so forth.

In [ ]:
cv = LeaveOneGroupOut()
decoder = Decoder(estimator='svc', mask=subject_data.mask_vt[0], 
                  standardize='zscore_sample', screening_percentile=100, 
                  scoring='accuracy', cv=cv)

# Compute the prediction accuracy for the different folds (i.e. session)
decoder.fit(fmri_sel, conditions_sel, groups=run_label)

decoder.cv_scores_['face']

What you see above is the classification accuracy for each of the validation runs. Notice that there are 12 accuracy scores here? These correspond to the 12 fMRI runs! We can calculate the average classification accuracy across all of these cross-validations, which will give an overall measure of accuracy.

In [ ]:
res_trs_vtc = print_class_accuracy(decoder)

Well, I think that is pretty impressive! What do you think? (Remember, this is classifying single frames/images of fMRI data!)


### Now, let's combine the code above into a function so that we can re-use this framework

It is generally better to us a function rather than copying and pasting the same code all over the place. For example, there is a smaller chance of error and it is more (human) readable. So, here let's get the pieces above into a workable function.

In [ ]:
def select_data(fmri_str, behavioral, conditions_arr):
    """
    Function for selecting data from a given conditions array.
    
    Parameters
    ----------
    fmri_str : string
        A string indicating the path and name of the fMRI data
    behavioral : pandas.DataFrame
        The behavioral conditions (here, expecting the Haxby data
        with the colums of labels and chunks)
    conditions_arr : a numpy.array
        A 1D numpy.array with the conditions to include
    
    Returns
    -------
    fmri_sel : the fMRI data from the selected frames
    conditions_sel : a numpy.array with the conditions labels
    run_sel : a numpy.array with the run labels
    """
    condition_mask = behavioral['labels'].isin(conditions_arr)
    conditions_sel = behavioral['labels'][condition_mask]
    run_sel = behavioral['chunks'][condition_mask]
    fmri_sel = index_img(fmri_str, condition_mask)
    return fmri_sel, conditions_sel, run_sel


def svm_loro_cv(fmri_data, behav_df, mask, conditions_arr):
    """
    Wrapper for running LORO cross validation with an SVM.
    
    Parameters
    ----------
    fmri_data : string
        A string indicating the path and name of the fMRI data
    behav_df : pandas.DataFrame
        The behavioral conditions (here, expecting the Haxby data
        with the columns of labels and chunks)
    conditions_arr : a numpy.array
        A 1D numpy.array with the conditions to include
    
    Returns
    -------
    Out : decoder
        Prints the decoding results and returns the fit nilearn decoder.
    """
    cv = LeaveOneGroupOut()
    fmri_sel, cond_sel, run_sel = select_data(fmri_data, behav_df, 
                                              conditions_arr)
    decoder = Decoder(estimator='svc', mask=mask, standardize='zscore_sample', 
                      screening_percentile=100, scoring='accuracy', cv=cv)
    decoder.fit(fmri_sel, cond_sel, groups=run_sel)
    avg_class_accuracy = print_class_accuracy(decoder)
    return decoder



Let's verify this gives us the same exact values as before:

In [ ]:
res = svm_loro_cv(fmri, tr_info, subject_data.mask_vt[0], np.array(('face', 'house')))
res.cv_scores_['face']

##### Looks like it worked!

### Now, we will want to see what happens if we restrict our analysis to specific regions of interest

#### How about looking at the house-selective voxels?

In [ ]:
res = svm_loro_cv(fmri, tr_info, subject_data.mask_house[0], 
                  np.array(('face', 'house')))

Quite similar to the overall accuracy throughout the ventral temporal cortex.

### Now, we will want to look at other categories: how selective is the "place" area?

Here, we will look at how well this region can classify other categories of stimuli: cat vs. shoe.

In [ ]:
res = svm_loro_cv(fmri, tr_info, subject_data.mask_house[0], 
                  np.array(('cat', 'shoe')))

#### Question 1

What does it mean that the "house-selective" region can classify between different categories of visual stimuli?

## Now, let's try running the decoder on the output from a general linear model (GLM)

The steps above were fitting / testing the classifier on individual fMRI frames (i.e., TRs; the images that took place during the viewing of images from different categories). However, these could each be relatively noisy and it is generally better to run a GLM to estimate activation to different trials/stimuli/etc. Below, we will slightly modify our code from last week to gather activity in response to each category for each run.

### First, let's set up our events DataFrame so that we can fit the models (GLMs)

Gather the event timing (onsets and durations), which we will use to set up our models (see below). The difference between tr_info (above) and events (below) is that the former contains information about what is happening for each frame of the fMRI data while the latter shows the exact times (in seconds) at which different events happen.

Note: the events.csv file in the line below is the file that you uploaded from Moodle into the directory with this notebook. If you get an error on this line, then it probably means that you need to upload this file.

In [ ]:
events = pd.read_csv('%s/events.csv' % datapath)
events

#### Extract the fMRI and event files into a list where the elements are the runs.

Now, we will want to separate the data based on the runs. Here, we will load the data into lists in which each element contains the data for one fMRI run. 

Note: hopefully the [x for x in y] type of syntax is familar from the earlier homeworks/readings, but this is basically a way that you can populate a list with a for loop in a single line of code. Here, it will loop over all of the runs and separate the data into a list. We will end up with lists with 12 elements: fmri_runs will have 12 runs of fMRI data and events_runs will have 12 runs of events/behavioral/timing data.

In [ ]:
run_nums = np.unique(events['run'])

fmri_runs = [index_img(fmri, tr_info['chunks'] == run_i) for run_i in run_nums]
events_runs = [events[events['run'] == run_i] for run_i in run_nums]

### Modify our code slightly from last week to output z-maps.

These files will create one z-score map for each category for each run. That is, instead of having 9 examples of each category per run (as above), we will end up with 1 example of each category per run.

In [ ]:
drift_model = 'polynomial'
poly_drift_order = 3
high_pass = 0.01
mask_img = subject_data.mask_vt[0]

In [ ]:
# set up empty lists in which we can store our information of interest --------
z_maps = []
task_label_glm = []
run_label_glm = []

# set up an array with all of the unique conditions (for looping below) -------
task_unique = np.unique(events['trial_type'])

# loop over all of the runs to run the GLM ------------------------------------
for run_i in np.arange(len(fmri_runs)):
    fmri_glm = FirstLevelModel(t_r=2.5, noise_model='ar1', standardize=False,
                               hrf_model='spm', drift_model=drift_model, 
                               drift_order=poly_drift_order, 
                               high_pass=high_pass, mask_img=mask_img)
    print(fmri_runs[run_i].shape)
    print(events_runs[run_i])
    fmri_glm = fmri_glm.fit(fmri_runs[run_i], events_runs[run_i])
    # extract z-statistics for each condition; i.e., our GLM-based patterns ---
    # and append the z-statistics, task labels, and run labels ----------------
    for condition_i in task_unique:
        print(condition_i)
        z_maps.append(fmri_glm.compute_contrast(condition_i))
        task_label_glm.append(condition_i)
        run_label_glm.append(run_i)
    plot_design_matrix(fmri_glm.design_matrices_[0])
    plt.show()

# convert the lists to numpy arrays -------------------------------------------
z_maps = np.array(z_maps)
task_label_glm = np.array(task_label_glm)
run_label_glm = np.array(run_label_glm)

### Here are a couple of functions that will help for the classification of different categories with different masks.

In [ ]:
def run_class_zmap(z_maps_arg, task_label_arg, run_label_arg, 
                   mask, categories):
    """
    Run classification analysis on z-maps with categories.
    
    Note: here, I am omitting z-scoring because we are already using
    the z-maps, thus we are effectively standardized within each run.
    
    Parameters
    ----------
    z_maps_arg : numpy.array with z-score maps
        A numpy.array with z-score maps from a GLM
    task_label_arg : a numpy.array of strings
        The task labels that correspond to the z-score maps
    run_label_arg : a numpy.array of int
        The run labels that correspond to the z-score maps
    mask : string
        A string indicating the path and name of the mask
    categories: numpy.array
        A numpy.array of the categories to include in the classification
    
    Returns
    -------
    Out : None
        Does not return anything but prints the results of decoding.
    """
    sel_mask = [x in categories for x in task_label_arg]
    z_maps_sel = z_maps_arg[sel_mask]
    task_sel = task_label_arg[sel_mask]
    run_sel = run_label_arg[sel_mask]
    decoder = Decoder(estimator='svc', mask=mask, standardize=False,
                      screening_percentile=100, cv=LeaveOneGroupOut())
    decoder.fit(z_maps_sel, task_sel, groups=run_sel)
    avg_acc = print_class_accuracy(decoder)
    return avg_acc


### Then, let's run the classification for faces vs. houses in VT with these z-score maps (from the GLMs)

In [ ]:
res_FvH_vtc = run_class_zmap(z_maps, task_label_glm, run_label_glm, 
                             mask=subject_data.mask_vt[0], 
                             categories=np.array(['face', 'house']))

Okay, 100% now! 

### What about classification in VT for other CATegories?

In [ ]:
res_CvS_vtc = run_class_zmap(z_maps, task_label_glm, run_label_glm,
                             mask=subject_data.mask_vt[0], 
                             categories=np.array(['cat', 'shoe']))

This is also looking purrfect! ;) 

### Let's try the same process for the "house" selective voxels

#### Faces vs. houses

In [ ]:
res_FvH_ppa = run_class_zmap(z_maps, task_label_glm, run_label_glm, 
                             mask=subject_data.mask_house[0], 
                             categories=np.array(['face', 'house']))

#### Cats vs. shoes

In [ ]:
# INPUT YOUR CODE HERE
res_CvS_ppa = 

### Let's try the same process for the "face" selective voxels

#### Faces vs. houses

In [ ]:
res_FvH_ffa = run_class_zmap(z_maps, task_label_glm, run_label_glm, 
                             mask=subject_data.mask_face[0], 
                             categories=np.array(['face', 'house']))

#### Cats vs. shoes

In [ ]:
# INPUT YOUR CODE HERE
res_CvS_ffa = 

### Let's create a figure with the results for the classification of Faces vs. "Places" and Cats vs. Shoes in the ventral temporal cortex as well as the face-selective voxels and the "place"-selective voxels

#### First, we need to get the data into a pandas DataFrame

In [ ]:
# get all of the classification accuracy into a numpy array -------------------
# here, we will multiply by 100 since percent accuracy is easier for plots ----
acc_all_data = np.array((res_FvH_vtc, res_FvH_ffa, res_FvH_ppa, 
                         res_CvS_vtc, res_CvS_ffa, res_CvS_ppa)) * 100

# create a numpy array with the classification categories ---------------------
classification_categories = np.append(np.repeat("Faces vs. Places", 3), 
                                      np.repeat("Cats vs. Shoes", 3))

# create a numpy array with the region of interest information (i.e., mask) ---
region_of_interest = np.array(("VTC", "FFA", "PPA", "VTC", "FFA", "PPA"))

# place it all into a DataFrame for easy plotting in seaborn ------------------
df = pd.DataFrame({"Classification accuracy" : acc_all_data, 
                   "Classification categories" : classification_categories, 
                   "Region of interest" : region_of_interest})

In [ ]:
df

#### Next, we can plot the data using seaborn

In [ ]:
g = sns.FacetGrid(df, col="Classification categories", hue="Region of interest", 
                  palette="colorblind", aspect=0.65)
g.map(sns.barplot, "Region of interest", "Classification accuracy", 
      order=["VTC", "FFA", "PPA"])
g.set_titles('{col_name}')
plt.savefig("classification_glm.pdf", bbox_inches="tight")

#### Question 2

How do the results from this GLM-based approach compare to the single-TR based approach above? For example, is classification accuracy generally better/worse/the same?

#### Question 3

If there were differences between single TRs and the GLM, what could potentially cause these differences?

#### Question 4

How do you interpret these findings? How do these results fit in with the activation analysis from last week? For example, similar to Question 1 above, do these results cause you to reinterpret your conclusions from last week (e.g., about potential modularity / specialization)? Why or why not? Please also discuss which theory or theories activation analysis vs. MVPA allow us to test.